In [23]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler , LabelEncoder , OneHotEncoder
import pickle

In [24]:
data = pd.read_csv('churn_modelling_mock.csv')

In [25]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,23882676,Smith,840,France,Female,36,8,55632.06,1,0,0,98692.59,1
1,2,21241085,Jones,419,Spain,Female,21,9,23820.59,3,0,0,100328.10,0
2,3,35705918,Brown,620,Germany,Female,59,4,183841.00,1,0,0,26725.66,0
3,4,71446290,Johnson,424,France,Male,42,7,125001.53,4,1,1,52497.78,1
4,5,90595402,Williams,406,France,Male,37,6,65544.36,4,0,1,40068.71,0


In [26]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

In [27]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,840,France,Female,36,8,55632.06,1,0,0,98692.59,1
1,419,Spain,Female,21,9,23820.59,3,0,0,100328.10,0
2,620,Germany,Female,59,4,183841.00,1,0,0,26725.66,0
3,424,France,Male,42,7,125001.53,4,1,1,52497.78,1
4,406,France,Male,37,6,65544.36,4,0,1,40068.71,0


In [28]:
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])

In [29]:
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,1.0,0.0,0.0
...,...,...,...
19995,0.0,1.0,0.0
19996,0.0,0.0,1.0
19997,0.0,1.0,0.0
19998,0.0,0.0,1.0


In [30]:
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,840,0,36,8,55632.06,1,0,0,98692.59,1,1.0,0.0,0.0
1,419,0,21,9,23820.59,3,0,0,100328.10,0,0.0,0.0,1.0
2,620,0,59,4,183841.00,1,0,0,26725.66,0,0.0,1.0,0.0
3,424,1,42,7,125001.53,4,1,1,52497.78,1,1.0,0.0,0.0
4,406,1,37,6,65544.36,4,0,1,40068.71,0,1.0,0.0,0.0


In [31]:
X = data.drop('EstimatedSalary', axis=1)
y = data['EstimatedSalary']

In [32]:
y.head()

0     98692.59
1    100328.10
2     26725.66
3     52497.78
4     40068.71
Name: EstimatedSalary, dtype: float64

In [33]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [34]:
scaler = StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [37]:
with open('salary_label_encoder_gender.pkl','wb') as file:
  pickle.dump(label_encoder,file)

with open('salary_onehot_encoder_geo.pkl','wb') as file:
  pickle.dump(onehot_encoder_geo,file)

with open('salary_scaler.pkl','wb') as file:
  pickle.dump(scaler,file)

In [40]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [42]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
  Dense(32, activation='relu'),
  Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [43]:
from tensorflow.keras.callbacks import EarlyStopping , TensorBoard
import datetime
log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [44]:
earlystopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

###Train the model

In [45]:
history = model.fit(
  X_train, y_train,
  validation_data=(X_test, y_test),
  epochs=100,
  callbacks=[earlystopping_callback , tensorboard_callback],
)

Epoch 1/100


500/500 [==============================] - 3s 2ms/step - loss: 8955926528.0000 - mae: 85946.9766 - val_loss: 8897063936.0000 - val_mae: 85593.6172
Epoch 2/100
500/500 [==============================] - 1s 2ms/step - loss: 8039679488.0000 - mae: 80377.0078 - val_loss: 7188457472.0000 - val_mae: 75003.8125
Epoch 3/100
500/500 [==============================] - 1s 1ms/step - loss: 5688015872.0000 - mae: 64813.9102 - val_loss: 4373372416.0000 - val_mae: 55504.8867
Epoch 4/100
500/500 [==============================] - 1s 1ms/step - loss: 3125921280.0000 - mae: 46135.3945 - val_loss: 2285723392.0000 - val_mae: 40004.9844
Epoch 5/100
500/500 [==============================] - 1s 1ms/step - loss: 1848179328.0000 - mae: 36891.2148 - val_loss: 1660519808.0000 - val_mae: 35574.7031
Epoch 6/100
500/500 [==============================] - 1s 1ms/step - loss: 1599852800.0000 - mae: 34861.0078 - val_loss: 1593115392.0000 - val_mae: 34756.6406
Epoch 7/100
500/500 [=======================

In [46]:
%load_ext tensorboard

In [48]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 808), started 0:00:27 ago. (Use '!kill 808' to kill it.)

In [49]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test MAE: {test_mae}")

125/125 [==============================] - 0s 918us/step - loss: 1576274944.0000 - mae: 34522.7930
Test Loss: 1576274944.0, Test MAE: 34522.79296875


In [50]:
model.save('regression_model.h5')

e:\AIML Practice\ANN Classification\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
